In [77]:
import roboticstoolbox as rtb
from mpl_toolkits.mplot3d.proj3d import transform
from roboticstoolbox import ERobot, ET, ETS
from scipy.spatial.transform import Rotation as R
from itertools import product
import numpy as np
from spatialmath import *


# Define the sequence of rotation axes for the joints
axes = ['z', 'x', 'z', 'x', 'z', 'x']
lengths = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
# Initialize an empty ETS

def create_robot(axes, lengths):
    
    # define inertia tensor for cylinder
    def inertia_tensor_cylinder(radius, height):
        # Inertia tensor for a solid cylinder about its central axis
        mass = np.pi * radius**2 * height
        I_xx = (1/12) * mass * (3 * radius**2 + height**2)
        I_yy = (1/12) * mass * (3 * radius**2 + height**2)
        I_zz = (0.5) * mass * radius**2
        return np.array([[I_xx, 0, 0],
                         [0, I_yy, 0],
                         [0, 0, I_zz]])
    
    
    transform_dict = {
    'z': ET.Rz,
    'x': ET.Rx,
    'y': ET.Ry
    }
    
    ets = ETS()
    for length, axis in zip(lengths, axes):
        ets *= transform_dict[axis]()
        ets *= ET.tz(length)
        
    robot = ERobot(ets, name='6-Link Robot Arm')
    
    for i in range(6):
        robot.links[i].r = [0, 0, 0.2]
        robot.links[i].I = inertia_tensor_cylinder(0.1, 0.4)
        robot.links[i].m = np.pi * 0.1**2 * 0.4
    
    return robot

my_robot = create_robot(axes, lengths)

T = SE3(0, 0.4, 0.4) * SE3.Rx(-np.pi*3/4)
mask = np.array([1, 1, 1, 1, 1, 0])

result = my_robot.ikine_LM(T, mask=mask, q0 = np.zeros(6))

print(my_robot.manipulability(result.q, method='yoshikawa'))
print(result)
#get rotation matrix from fkine
my_robot.fkine(result.q).R @ np.array([0, 0, 1])

0.018324829587305868
IKSolution: q=[0, 0.3577, 0, -2.012, 0, -0.7021], success=True, iterations=7, searches=1, residual=1.98e-07


array([ 0.        ,  0.70710678, -0.70710678])

In [ ]:
initial = np.array([0, 100, 0])
z_range = np.linspace(np.radians(-70), np.radians(70), 30) #left/right rotation
y_range = np.linspace(np.radians(-40), np.radians(40), 30) #neck lateral bending
x_range = np.linspace(np.radians(-60), np.radians(60), 30) #neck extension/ flexion

prod = np.array(list(product(z_range, y_range, x_range)))

rotations = R.from_euler('zyx', prod, degrees=False)
rotated = rotations.apply(initial)

zipper = zip(rotations, rotated)

In [23]:
from tqdm import tqdm

Nfeval = 1

def callbackF(Xi, *rest1, **rest2):
    global Nfeval
    print(Nfeval, Xi, rest1, rest2)
    Nfeval += 1

axes = ['z', 'y', 'z', 'y', 'z', 'y']  # Standard configuration

transform_dict = {
    'z': ET.Rz,
    'x': ET.Rx,
    'y': ET.Ry
}

def manipulability_objective(link_lengths):
    # Build the robot model with the given link lengths
    ets = ETS()
    for length, axis in zip(link_lengths, axes):
        ets *= transform_dict[axis]()
        ets *= ET.tz(length)
    robot = ERobot(ets, name='6-Link Robot Arm')    
    # Solve IK
    ik_result = robot.ikine_LM(T, q0=np.zeros(6))
    if not ik_result.success:
        return 1e6
    q = ik_result.q
    
    # Compute Asada manipulability
    manipulability = robot.manipulability(q, method='yoshikawa')
    
    return -manipulability

from scipy.optimize import minimize, differential_evolution, basinhopping

initial_link_lengths = [0.2] * 6
bounds = [(0.1, 0.7)] * 6

# result1 = differential_evolution(manipulability_objective, bounds, maxiter=1000, popsize=15, callback=callbackF)
# result2 = basinhopping(manipulability_objective, initial_link_lengths, niter=1, T=0.1, stepsize=0.1, callback=callbackF)

result = minimize(manipulability_objective, initial_link_lengths, bounds=bounds, method='L-BFGS-B', options={'maxiter': 1000}, callback=callbackF)


# print(f"Optimized Link Lengths: {result1.x}")
# print(f"Optimized Link Lengths: {result2.x}")
# print(f"Maximum Asada Manipulability: {-result1.fun}")
# print(f"Maximum Asada Manipulability: {-result2.fun}")

print(f"Optimized Link Lengths: {result.x}")
print(f"Maximum Asada Manipulability: {-result.fun}")

1 [0.26844375 0.26844375 0.26844375 0.22916523 0.2271932  0.20213457] () {}
2 [0.26844375 0.26844375 0.26844375 0.22916523 0.2271932  0.20213457] () {}
Optimized Link Lengths: [0.26844375 0.26844375 0.26844375 0.22916523 0.2271932  0.20213457]
Maximum Asada Manipulability: 0.11641707578545891


In [53]:
from sympy import *
rot=diag(1, -1, -1)
vec=Matrix([0, 0, 1])

rot@vec


Matrix([
[ 0],
[ 0],
[-1]])